In [1]:
from ipynb.fs.defs.functions import htmltotext, some_actions, query_url, send_query, save_data, check_new_articles

In [15]:
from time import gmtime, strftime
from datetime import datetime, timedelta
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import nltk
import json
import time
import random
import traceback
import pickle

browser = webdriver.Chrome(  executable_path = "C:\\Chrome\\chromedriver" )
action  = webdriver.ActionChains(browser)

In [3]:
def change_browser():
    print("reload ")
    global browser, action
    browser.close()
    browser = webdriver.Chrome(  executable_path = "C:\\Chrome\\chromedriver" )
    browser.set_window_size(1920, 1080)
    action  = webdriver.ActionChains(browser)

# Process

In [4]:
def confirm_not_robot():
    print("error for confirm")

In [19]:
def get_article_urls_from_html( soup, articles ):

    articles_url_name = {}
    
    
    for article in articles:
        # with href we get the position trend
        if "?" in article['href']:
            article['href'] = article['href'].split("?")[0]
        href  = article['href']
        title = article.get_text() 
        articles_url_name[ href ] = title
        
    return articles_url_name

In [6]:
def get_urls_from_cotegory( url, cnt=None ): # cnt for recursion count

    html = query_url(url, browser, wait_time)
    soup = BeautifulSoup(html, 'html.parser')
    
    # a   with href articles
    articles = [ article for article in soup.find_all( "a" ) if "/articles/" in article["href"] ]
    
    # if in captcha page
    if   len( articles ) == 0 and cnt == None :
        print(url)
        confirm_not_robot()
        return get_urls_from_cotegory( url, cnt=1 )
    elif len( articles ) == 0 and cnt == 1:
        return {}
    
    
    # get urls and article names from html    
    return get_article_urls_from_html( soup, articles )

In [7]:
def get_article_content( url, cnt=None ):
    
    html = query_url(url, browser, wait_time)
    soup = BeautifulSoup(html, 'html.parser')
    
    compare_element = soup.find_all( "div",  attrs={"class" : "wsj-snippet-body"} )
    compare_element += soup.find_all("div", attrs={"class" : "article-content"})
    
    if  len( compare_element ) == 0 and cnt == None :
        print(url)
        confirm_not_robot()
        return get_article_content( url, cnt=1 )
            
    elif len( compare_element ) == 0 and cnt == 1:
        return "", ""
    
    # get free content ( group )
    content  = compare_element

    text_cont = " ".join( [ element.get_text() for element in content ] )  
    text_cont = htmltotext( text_cont )
    
    # get article time
    time = soup.find( "time" ).get_text()
    
    return text_cont, time

In [8]:
def main_process( category, category_articles_data, sleep_between_url , url, url_server ):
    
    # check new articles    
    articles_url_name_new = get_urls_from_cotegory( url )
    articles_old_urls = category_articles_data["url"].values
    new_articles_urls = check_new_articles( list( articles_url_name_new.keys() ) , articles_old_urls )
    
    # if appear new article. get content
    for article_url in new_articles_urls:
        
        text_cont, time_art = get_article_content( article_url )
        
        # add new article data
        new_data = {
            "url"      : article_url,
            "title"    : articles_url_name_new[article_url],
            "text"     : text_cont,
            "date"     : time_art,
            "category" : category
        }
        # send data to server
        try:
            send_query( url_server, new_data )
        except:
            pass
        
        category_articles_data = category_articles_data.append( new_data, ignore_index=True )
        # each 5 article pickle
        if category_articles_data.shape[0] % 10 == 0:
            pickle.dump( category_articles_data , open(pickle_obj_path,"wb")) 
        # if article counts > 300, save 150 
        if category_articles_data.shape[0] > 300:
            category_articles_data = save_data( category_articles_data, df_save_path )
            pickle.dump( category_articles_data , open(pickle_obj_path,"wb")) 
        
        time.sleep( sleep_between_url )
    pickle.dump( category_articles_data , open(pickle_obj_path,"wb")) 

In [9]:
categorys = [ "/markets/", "/business/", "/economy/"]
df_columns = ["url", "title", "text", "date", "category"]

url_server = "http://localhost:9000/"
source_url = "https://www.wsj.com/news"
pickle_obj_path = "wsj_data.pkl"
df_save_path = "./WSJ/wsj_data"

sleep_between_url = 4
sleep_between_cat = 4
wait_time = 3

In [13]:
category_articles_data = pd.DataFrame( columns = df_columns )

In [ ]:
category_articles_data = pickle.load(open(pickle_obj_path, "rb"))

In [ ]:
while True:

    for category in categorys:

        url = source_url + category
        category_articles_data = pickle.load(open(pickle_obj_path, "rb"))
        
        try:
            main_process( category, category_articles_data, sleep_between_url, url, url_server )
        except Exception as e:
            print( str(e) )
            change_browser()
        
        time.sleep(sleep_between_cat)

https://www.wsj.com/news/business/
error for confirm
'href'
reload 
'href'
reload 
'href'
reload 
'href'
reload 
https://www.wsj.com/news/economy/
error for confirm
https://www.wsj.com/news/markets/
error for confirm
'href'
reload 
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

'href'
reload 
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

'href'
reload 
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

'href'
reload 
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

'href'
reload 
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

'href'
reload 
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

'href'
reload 
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

'href'
reload 
Message: unknown error: ne

https://www.wsj.com/news/economy/
error for confirm
https://www.wsj.com/news/economy/
error for confirm
https://www.wsj.com/news/markets/
error for confirm
https://www.wsj.com/news/economy/
error for confirm
https://www.wsj.com/news/economy/
error for confirm
https://www.wsj.com/news/economy/
error for confirm
https://www.wsj.com/news/business/
error for confirm
https://www.wsj.com/news/markets/
error for confirm
https://www.wsj.com/news/economy/
error for confirm
Message: timeout: Timed out receiving message from renderer: 3.000
  (Session info: chrome=92.0.4515.159)

reload 
Message: timeout: Timed out receiving message from renderer: 3.000
  (Session info: chrome=92.0.4515.159)

reload 
https://www.wsj.com/news/markets/
error for confirm
Message: timeout: Timed out receiving message from renderer: 3.000
  (Session info: chrome=92.0.4515.159)

reload 
https://www.wsj.com/news/economy/
error for confirm
Message: timeout: Timed out receiving message from renderer: 3.000
  (Session info

In [ ]:
category_articles_data["url"].values[0]